In [1]:
import cv2, pickle, platform, gc, numpy as np
import os
from tkinter import Tk, filedialog
from PIL import Image, ImageFont, ImageDraw

In [2]:
ROI_SIZE       = 300            # 중앙 사각형 픽셀 수
FRAME_W, FRAME_H = 1280, 720    # 웹캠 해상도
FONT_SIZE      = 24
PREVIEW_SCALE  = 6              # 28×28 → 168×168 로 확대

In [3]:
def relu(x): return np.maximum(0, x)

class Dense:
    def __init__(self, i, o):
        self.W = np.empty((i, o), np.float32)
        self.b = np.empty(o,      np.float32)
    def __call__(self, x): return x @ self.W + self.b

class MLP:
    def __init__(self):
        self.fc1 = Dense(784, 128)
        self.fc2 = Dense(128, 64)
        self.fc3 = Dense(64,  32)
        self.out = Dense(32,  10)
        self.layers = [self.fc1, self.fc2, self.fc3, self.out]
    def forward(self, x):
        x = relu(self.fc1(x)); x = relu(self.fc2(x))
        x = relu(self.fc3(x)); logits = self.out(x)
        return logits, None

In [4]:
target_model = "mnist_nn_model.pkl"

if not os.path.exists(target_model):
    print(f"'{target_model}' 파일이 없습니다. 파일을 선택해주세요...")

    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)

    selected_file = filedialog.askopenfilename(
        title="mnist_nn_model.pkl 파일 선택",
        filetypes=[("Pickle Files", "*.pkl"), ("All Files", "*.*")]
    )
    root.destroy()
    
    if selected_file:
        target_model = selected_file
        print(f"선택된 파일: {target_model}")
    else:
        print("파일이 선택되지 않았습니다. 프로그램을 종료합니다.")
        raise SystemExit("프로그램 종료: 모델 파일 미선택")

with open(target_model, "rb") as f:
    model = pickle.load(f)

sys_name = platform.system() 

font_path = {
    "Windows": "C:/Windows/Fonts/malgun.ttf",
    "Darwin":  "/Library/Fonts/AppleGothic.ttf"
}.get(sys_name, "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc")

try:
    font = ImageFont.truetype(font_path, FONT_SIZE)
except IOError:
    print("폰트 미탑재 → 기본 폰트 사용(한글 미지원)")
    font = ImageFont.load_default()

In [5]:
def softmax(z):
    e = np.exp(z - np.max(z, axis=1, keepdims=True))
    return e / np.sum(e, axis=1, keepdims=True)

In [15]:
def preprocess(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # ── 1) contrast stretching ──────────────────────
    p2, p98 = np.percentile(gray, (2, 98))
    if p98 - p2 < 5:                       # 전역 대비 너무 낮으면 fallback
        p2, p98 = 0, 255
    gray_cs = np.clip((gray - p2) * 255.0 / (p98 - p2), 0, 255).astype(np.uint8)

    # ── 2) 단일 임계값 (밝으면 255) ──────────────────
    _, bw = cv2.threshold(gray_cs, 160, 255, cv2.THRESH_BINARY)

    # ── 3) 가장 큰 컨투어(숫자) bbox ────────────────
    cnts, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        cnts = [np.array([[(0,0)],[(27,0)],[(27,27)],[(0,27)]], np.int32)]
    c = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)

    # ROI 추출 (+20 % 여백)
    margin = int(0.2 * max(w, h))
    gh, gw = gray.shape
    x0, y0 = max(x - margin, 0), max(y - margin, 0)
    x1, y1 = min(x + w + margin, gw - 1), min(y + h + margin, gh - 1)
    roi = gray_cs[y0:y1, x0:x1]

    # 정사각 패딩 → 28×28
    h2, w2 = roi.shape
    m = max(h2, w2)
    pad = np.full((m, m), 0, np.uint8)          # 검은 배경
    pad[(m - h2)//2:(m - h2)//2 + h2,
        (m - w2)//2:(m - w2)//2 + w2] = roi
    img28 = cv2.resize(pad, (28, 28), interpolation=cv2.INTER_AREA)

    # 0–1 스케일 & 자동 반전(배경이 어두우면 뒤집기)
    img = img28.astype(np.float32) / 255.0
    if img.mean() > 0.5:                # 배경 < 숫자 ⇒ 반전
        img = 1.0 - img

    vec = img.reshape(-1)               # 784-길이 벡터
    x_vec = (vec - MEAN) / STD
    return img28, x_vec[np.newaxis, :]

In [ ]:
cap = cv2.VideoCapture(0)
# 요청
cap.set(cv2.CAP_PROP_FRAME_WIDTH,  FRAME_W)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_H)
if not cap.isOpened():
    raise RuntimeError("웹캠을 열 수 없습니다.")

real_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
real_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

cx, cy = real_w // 2, real_h // 2
half   = ROI_SIZE // 2
print("실행 중…  (ESC 또는 x 로 종료)")

while True:
    ret, frame = cap.read()
    key = cv2.waitKey(1) & 0xFF
    if not ret or key in (27, ord('x')):
        break

    roi_frame = frame[cy - half: cy + half, cx - half: cx + half]
    img28, x_input = preprocess(roi_frame)

    logits, _ = model.forward(x_input)
    prob = softmax(logits)
    idx  = int(np.argmax(prob))
    conf = float(prob[0, idx])
    label = f"예측: {idx} ({conf*100:.1f}%)"

    # 시각화
    # 사각형
    cv2.rectangle(frame, (cx - half, cy - half), (cx + half, cy + half),
                  (0, 255, 0), 2)

    # 전처리 미리보기
    preview = cv2.resize(255 - img28, (28*PREVIEW_SCALE, 28*PREVIEW_SCALE),
                         interpolation=cv2.INTER_NEAREST)
    pv_h, pv_w = preview.shape
    frame[10:10+pv_h, FRAME_W - pv_w - 10: FRAME_W - 10] = \
        cv2.cvtColor(preview, cv2.COLOR_GRAY2BGR)

    # 텍스트
    rgb  = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil  = Image.fromarray(rgb)
    draw = ImageDraw.Draw(pil)
    draw.text((10, 10), label, font=font, fill=(0, 0, 0))
    disp = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
    cv2.imshow("MNIST Cam (ROI + Preview)", disp)

    del pil, draw, rgb; gc.collect()

cap.release()
cv2.destroyAllWindows()